In [19]:
# !jupyter nbconvert --to script AFL.ipynb

Traceback (most recent call last):
  File "/Users/conrad/miniconda3/envs/conrad/bin/jupyter", line 11, in <module>
    sys.exit(main())
  File "/Users/conrad/miniconda3/envs/conrad/lib/python3.6/site-packages/jupyter_core/command.py", line 285, in main
    command = _jupyter_abspath(subcommand)
  File "/Users/conrad/miniconda3/envs/conrad/lib/python3.6/site-packages/jupyter_core/command.py", line 125, in _jupyter_abspath
    'Jupyter command `{}` not found.'.format(jupyter_subcommand)
Exception: Jupyter command `jupyter-nbconvert` not found.


In [2]:
# !pip install pyomo
# !pip install --upgrade --user ortools
# !apt-get install -y glpk-utils
# !sudo apt-get update -y
# !sudo apt-get install -y coinor-cbc

In [1]:
# !pip install -U git+https://github.com/coin-or/pulp
# !sudo pulptest
# !pip install pandas

In [49]:
def test1(test_df):
    assert len(test_df[test_df['Name'].isna()])==0
    for r in test_df['Round'].unique():
        names = test_df[test_df['Round']==r]['Name'].unique()
        try:    
            assert len(test_df[~(test_df['Name'].isin(names))])==0
        except AssertionError:
            # print(r)
            # print(test_df[~(test_df['Name'].isin(names))])
            assert len(test_df[~(test_df['Name'].isin(names))])==0

In [55]:
import numpy as np
import pandas as pd
from pulp import *
full_player_scores = pd.read_csv('full_players.csv', index_col=0)
full_player_scores.dropna(subset=['Name'])
full_player_scores = full_player_scores.reset_index(drop=True)
full_player_scores['Score'] = pd.to_numeric(full_player_scores['Score'], errors='coerce')

# testfilter
full_player_scores = full_player_scores[full_player_scores['Round']<4]
# players_to_keep = ["Adam Tomlinson", "Max Gawn"]
# players_to_keep = full_player_scores['Name'].unique()[:100]
# full_player_scores = full_player_scores[full_player_scores['Name'].isin(players_to_keep)]


full_player_scores_2021 = full_player_scores[full_player_scores['Year']==2021]
full_player_scores = full_player_scores[full_player_scores['Year']==2020]

# Only keep players who are playing this season
full_player_scores = full_player_scores[full_player_scores['Name'].isin(full_player_scores_2021['Name'].unique())].dropna(how='all').dropna(how='all')

df = full_player_scores.copy()

df1 = df[['Name','Team','Position','Price', 'Team 1','Year']].drop_duplicates()
df2 = df[['Round']].drop_duplicates()
df1['key'] = 0
df2['key'] = 0
df = df1.merge(df2, how='outer', on=['key'])
df = df.sort_values('Round')
df = df.drop('key', axis=1)
full_player_scores = full_player_scores.merge(df, on=['Name', 'Team', 'Position','Round','Price', 'Team 1','Year'],how='right').reset_index(drop=True)
try:
    test1(full_player_scores)
except AssertionError:
    print('WARNING: failed a test')


player_num_games_played = full_player_scores[full_player_scores['Score']>0].groupby('Name')['Team 2'].transform('count')

opponent_average_score = full_player_scores.groupby('Team 2')['Score'].transform('mean')
player_average_score = full_player_scores.groupby('Name')['Score'].transform('mean')
player_average_value = player_average_score/full_player_scores['Price']

na_locs = full_player_scores['Score'].isna()

full_player_scores.loc[na_locs,'Score'] = ((opponent_average_score+player_average_score)/2).loc[na_locs]

na_locs = full_player_scores['Score'].isna()

full_player_scores.loc[na_locs,'Score'] = (player_average_score).loc[na_locs]

RU_Players = full_player_scores.copy()

assert list(full_player_scores['Year'].unique())==[2020]

RU_Players = RU_Players.sort_values('Score',ascending=False)

RU_Players = full_player_scores.copy()

assert list(full_player_scores['Year'].unique())==[2020]

RU_Players = RU_Players.sort_values('Score',ascending=False)
RU_Players['id'] = "R:"+RU_Players['Round'].astype(str)+"_P:"+RU_Players['Name']

# Only take players who played more than 5 games
# players_to_keep = player_num_games_played[player_num_games_played>5].index.values
# RU_Players = RU_Players.reindex(index=players_to_keep).dropna(how='all')

# Only take players in top quater of average values (score/price)
player_best_values = player_average_value.sort_values(ascending=False)[:int(len(player_average_value)/4)]
RU_Players = RU_Players.reindex(index=player_best_values.index.values).dropna(how='all') 

try:
    test1(RU_Players)
except AssertionError:
    print('WARNING: failed a test')

RU_Players.groupby(['Round','Position']).count()

Team 1  Team 2  Year  Score  Name  Team  Price  id
Round Position                                                    
1     DE            42      42    42     42    42    42     42  42
      FO            58      58    58     58    58    58     58  58
      MI            53      53    53     53    53    53     53  53
      RU            10      10    10     10    10    10     10  10
2     DE            42      42    42     42    42    42     42  42
      FO            57      57    57     57    57    57     57  57
      MI            53      53    53     53    53    53     53  53
      RU            10      10    10     10    10    10     10  10
3     DE            42      32    42     42    42    42     42  42
      FO            57      51    57     57    57    57     57  57
      MI            53      46    53     53    53    53     53  53
      RU            10      10    10     10    10    10     10  10

In [56]:
player_contraints = {}
prob = LpProblem("aflProblem", LpMaximize)
overall_score = LpVariable('OverallScore',0)
for p_id,score in zip(RU_Players['id'].iteritems(),RU_Players['Score'].iteritems()):
  p_id = p_id[1]
  player_contraints[p_id] = LpVariable(p_id, 0, 1, cat='Binary')

player_contraints = LpVariable.dicts("PC", player_contraints, 0, 1, cat='Binary')
prob += lpSum([player_contraints[p_id[1]]*score[1] for p_id,score in zip(RU_Players['id'].iteritems(),RU_Players['Score'].iteritems())]), "Total score is maximized"

# START TRANFERS
RU_Players_prev_round = RU_Players
RU_Players_prev_round['Round_prev'] = RU_Players_prev_round['Round'] - 1

RU_transfers = RU_Players[['id','Position','Round','Name']].merge(RU_Players_prev_round[['id','Position','Round_prev','Name']],left_on=['Position','Round'],right_on=['Position','Round_prev'], suffixes = ('_prev','_next'))

RU_transfers['Transfer'] = RU_transfers['id_prev']+'-'+RU_transfers['id_next']
RU_transfers=RU_transfers[['id_prev',	'Round',	'Name_prev',	'id_next',	'Round_prev',	'Name_next',	'Transfer']].drop_duplicates()

transfer_contraints = {}

# Define transfer itermediaries
for i,t_id in RU_transfers['Transfer'].iteritems():
  transfer_contraints[t_id] = LpVariable(t_id, 0, 1, cat='Binary')

transfer_contraints = LpVariable.dicts("TC", transfer_contraints, 0, 1, cat='Binary')

# map rounds end to itermediary
for prev_player,trans in RU_transfers.groupby(['id_prev'])['Transfer'].apply(list).iteritems():
  prob += lpSum([transfer_contraints[t_id] for t_id in trans]) == player_contraints[prev_player], f"Previous player equals transfer intermediatary for {prev_player} equals {trans}"

# map intermediary to next round
for next_player,trans in RU_transfers.groupby(['id_next'])['Transfer'].apply(list).iteritems():
  prob += lpSum([transfer_contraints[t_id] for t_id in trans]) == player_contraints[next_player], f"Next player equals transfer intermediatary for {next_player} equals {trans}"

for round, transfers in RU_transfers[RU_transfers['Name_prev'] != RU_transfers['Name_next']][['Round','Transfer']].drop_duplicates().groupby(['Round'])['Transfer'].apply(list).iteritems():
  prob += lpSum([transfer_contraints[t_id] for t_id in transfers]) <= 4, f"Round: {round}, has less than or equal to 4 transfers"
  print(f'{[transfer_contraints[t_id] for t_id in transfers]}) <= 4')
# END TRANFERS

# START money contraint
money = 13 * 10**6
for round,player in RU_Players[['id','Price','Round']].drop_duplicates().groupby(['Round'])['id'].apply(list).iteritems():
  prob += lpSum([player_contraints[p_id]*RU_Players[RU_Players['id']==p_id]['Price'].values[0] for p_id in player]) <= money, f"Round: {round}, has less than ${money}"
# END money contraint

#START Contrain each player to a single position per round
RU_Players_Position_Casting = RU_Players.copy()
RU_Players_Position_Casting['pl_pos_id'] = (RU_Players['id']+'_'+RU_Players['Position'])

Players_Position_Casting_Series = RU_Players_Position_Casting[['id','pl_pos_id','Round']].drop_duplicates().groupby(['id','Round'])['pl_pos_id'].apply(list).iteritems()

Players_Position_Casting = {}
for i,r in RU_Players_Position_Casting[['pl_pos_id']].drop_duplicates().iterrows():
  Players_Position_Casting[r['pl_pos_id']] = LpVariable(r['pl_pos_id'], 0, 1, cat='Binary')

Players_Position_Casting = LpVariable.dicts("Players_Position_Casting", Players_Position_Casting, 0, 1, cat='Binary')

for (p_id,round), pl_pos_ids in Players_Position_Casting_Series:
    player_possible_positions = [Players_Position_Casting[pl_pos_id] for pl_pos_id  in pl_pos_ids] 
    prob += lpSum(player_possible_positions) <= player_contraints[p_id], f"Must have only have equal to {p_id} (0 or 1) of {player_possible_positions} positions in round {round}"
#END Contrain each player to a single position per round

# START max players from each position
# allowed_holds_per_position = {'DE': 8, "MI" : 10, 'RU' : 3, 'FO':9}
allowed_holds_per_position = {'DE': 1, "MI" : 1, 'RU' : 1, 'FO':1}

print('START max players from each position')
for (position,round), pl_pos_ids in RU_Players_Position_Casting[['pl_pos_id','Position','Round']].drop_duplicates().groupby(['Position', 'Round'])['pl_pos_id'].apply(list).iteritems():
  prob += lpSum([Players_Position_Casting[pl_pos_id] for pl_pos_id in pl_pos_ids]) == allowed_holds_per_position[position], f"Position: {position}, has less than {allowed_holds_per_position[position]} in round {round}"
  print('\n')

# END max players from each position

print('done with pre-work')

R:2_P:Easton_Wood_R:3_P:Will_Hamill, TC_R:2_P:Easton_Wood_R:3_P:Hayden_Crozier, TC_R:2_P:Easton_Wood_R:3_P:Dylan_Roberton, TC_R:2_P:Easton_Wood_R:3_P:Aliir_Aliir, TC_R:2_P:Easton_Wood_R:3_P:Jake_Carlisle, TC_R:2_P:Easton_Wood_R:3_P:Jay_Lockhart, TC_R:2_P:Easton_Wood_R:3_P:Wayne_Milera, TC_R:2_P:Easton_Wood_R:3_P:Christian_Salem, TC_R:2_P:Easton_Wood_R:3_P:Blake_Hardwick, TC_R:2_P:Easton_Wood_R:3_P:Nick_Haynes, TC_R:2_P:Michael_Hurley_R:3_P:Michael_Hibberd, TC_R:2_P:Michael_Hurley_R:3_P:Connor_Budarick, TC_R:2_P:Michael_Hurley_R:3_P:Brandon_Starcevich, TC_R:2_P:Michael_Hurley_R:3_P:Sam_Frost, TC_R:2_P:Michael_Hurley_R:3_P:Noah_Answerth, TC_R:2_P:Michael_Hurley_R:3_P:Trent_McKenzie, TC_R:2_P:Michael_Hurley_R:3_P:Mason_Redman, TC_R:2_P:Michael_Hurley_R:3_P:Ryan_Gardner, TC_R:2_P:Michael_Hurley_R:3_P:Joel_Smith, TC_R:2_P:Michael_Hurley_R:3_P:Sam_Docherty, TC_R:2_P:Michael_Hurley_R:3_P:Kaiden_Brand, TC_R:2_P:Michael_Hurley_R:3_P:Alex_Keath, TC_R:2_P:Michael_Hurley_R:3_P:Sam_Taylor, TC_R:2_P

In [57]:
try:
    solver = getSolver('COIN_CMD', timeLimit=200, msg=True,gapRel = 0.2)
    prob.solve(solver)
except:
    print('USING INBUILT SOLVER!!  \n'*100)
    solver = getSolver('PULP_CBC_CMD')
    prob.solve(solver)

USING INBUILT SOLVER!!  
USING INBUILT SOLVER!!  
USING INBUILT SOLVER!!  
USING INBUILT SOLVER!!  
USING INBUILT SOLVER!!  
USING INBUILT SOLVER!!  
USING INBUILT SOLVER!!  
USING INBUILT SOLVER!!  
USING INBUILT SOLVER!!  
USING INBUILT SOLVER!!  
USING INBUILT SOLVER!!  
USING INBUILT SOLVER!!  
USING INBUILT SOLVER!!  
USING INBUILT SOLVER!!  
USING INBUILT SOLVER!!  
USING INBUILT SOLVER!!  
USING INBUILT SOLVER!!  
USING INBUILT SOLVER!!  
USING INBUILT SOLVER!!  
USING INBUILT SOLVER!!  
USING INBUILT SOLVER!!  
USING INBUILT SOLVER!!  
USING INBUILT SOLVER!!  
USING INBUILT SOLVER!!  
USING INBUILT SOLVER!!  
USING INBUILT SOLVER!!  
USING INBUILT SOLVER!!  
USING INBUILT SOLVER!!  
USING INBUILT SOLVER!!  
USING INBUILT SOLVER!!  
USING INBUILT SOLVER!!  
USING INBUILT SOLVER!!  
USING INBUILT SOLVER!!  
USING INBUILT SOLVER!!  
USING INBUILT SOLVER!!  
USING INBUILT SOLVER!!  
USING INBUILT SOLVER!!  
USING INBUILT SOLVER!!  
USING INBUILT SOLVER!!  
USING INBUILT SOLVER!!  


In [60]:
prob

ick <= 1 Integer
0 <= TC_R:2_P:Wayne_Milera_R:3_P:Darcy_Byrne_Jones <= 1 Integer
0 <= TC_R:2_P:Wayne_Milera_R:3_P:David_Astbury <= 1 Integer
0 <= TC_R:2_P:Wayne_Milera_R:3_P:Dylan_Roberton <= 1 Integer
0 <= TC_R:2_P:Wayne_Milera_R:3_P:Easton_Wood <= 1 Integer
0 <= TC_R:2_P:Wayne_Milera_R:3_P:Hayden_Crozier <= 1 Integer
0 <= TC_R:2_P:Wayne_Milera_R:3_P:Jack_Bowes <= 1 Integer
0 <= TC_R:2_P:Wayne_Milera_R:3_P:Jake_Carlisle <= 1 Integer
0 <= TC_R:2_P:Wayne_Milera_R:3_P:James_Harmes <= 1 Integer
0 <= TC_R:2_P:Wayne_Milera_R:3_P:Jay_Lockhart <= 1 Integer
0 <= TC_R:2_P:Wayne_Milera_R:3_P:Jeremy_Howe <= 1 Integer
0 <= TC_R:2_P:Wayne_Milera_R:3_P:Joel_Smith <= 1 Integer
0 <= TC_R:2_P:Wayne_Milera_R:3_P:Jordan_Clark <= 1 Integer
0 <= TC_R:2_P:Wayne_Milera_R:3_P:Kaiden_Brand <= 1 Integer
0 <= TC_R:2_P:Wayne_Milera_R:3_P:Mason_Redman <= 1 Integer
0 <= TC_R:2_P:Wayne_Milera_R:3_P:Michael_Hibberd <= 1 Integer
0 <= TC_R:2_P:Wayne_Milera_R:3_P:Michael_Hurley <= 1 Integer
0 <= TC_R:2_P:Wayne_Milera_R:

In [59]:
for i,v in transfer_contraints.items():

    if v.value() != 0:
        # print(v.value())
        # print(i)
        last_name = i[i.find(' ')+1: i.find('-')]
        last_letters = i[-len(last_name):]
        # print(last_name)
        # print(last_letters)
        # print
        if not last_name == last_letters:
            print(i)

R:2_P:Steven Motlop-R:3_P:Isaac Heeney
R:2_P:Sam Sturt-R:3_P:Ben King
R:1_P:Ben Cunnington-R:2_P:Sebastian Ross
R:1_P:Jack Viney-R:2_P:Matt Rowell
R:2_P:Sebastian Ross-R:3_P:Callan Ward
R:2_P:Tom Clurey-R:3_P:Sam Taylor
R:1_P:Mason Redman-R:2_P:Connor Budarick
R:1_P:Ryan Gardner-R:2_P:Brandon Starcevich


In [30]:
results = []
for name, player_position in Players_Position_Casting.items():
    if player_position.value() != 0:
        results.append([name,1])
Players_selected = pd.DataFrame(results, columns=['pl_pos_id', 'is_selected'])
RU_Players_Position_Casting = RU_Players_Position_Casting.merge(Players_selected, on = ['pl_pos_id'])
RU_Players_Position_Casting.to_csv('solution.csv')
print(RU_Players_Position_Casting.groupby(['Round','Position']).sum()['is_selected'])

Round  Position
1      RU          1
2      RU          1
3      RU          1
Name: is_selected, dtype: int64


In [31]:
RU_Players_Position_Casting.sort_values(['Round'])

,Round,Team 1,Team 2,Year,Score,Name,Team,Position,Price,id,Round_prev,pl_pos_id,is_selected
0,1,Magpies,Bulldogs,2020,114.0,Brodie Grundy,Magpies,RU,867000,R:1_P:Brodie Grundy,0,R:1_P:Brodie Grundy_RU,1
1,2,Magpies,Tigers,2020,82.0,Brodie Grundy,Magpies,RU,867000,R:2_P:Brodie Grundy,1,R:2_P:Brodie Grundy_RU,1
2,3,Magpies,Saints,2020,99.0,Brodie Grundy,Magpies,RU,867000,R:3_P:Brodie Grundy,2,R:3_P:Brodie Grundy_RU,1


In [32]:
for round in RU_Players_Position_Casting['Round'].unique():
    assert RU_Players_Position_Casting[RU_Players_Position_Casting['Round']==round]['is_selected'].sum()